In [142]:
downloads_RDD = spark.read.csv("file:///home/jawad/assigment1/*.gz", inferSchema = True, header = True)

In [143]:
#downloads_RDD=downloads_RDD.where(downloads_RDD.r_os!="NA")
downloads_RDD.createOrReplaceTempView('packages')

In [292]:
def insert_into_cassandra(query,table_name):
    query.write.format("org.apache.spark.sql.cassandra").\
    options(table=table_name, keyspace="assignment02").save(mode="append")

In [294]:
#---------------------------------    Question# 1   ---------------------------------------------------------
#     Show number of downloads for package ggplot2 and dplyr.
#------------------------------------------------------------------------------------------------------------
query_results = spark.sql('SELECT package,count(*) as count FROM packages WHERE package\
                            IN("ggplot2", "dplyr") GROUP BY package')
insert_into_cassandra(query_results,"question1")

In [295]:
#---------------------------------    Question# 2   ---------------------------------------------------------
#     Total number of downloads by each Operating System (group similar ones).
#------------------------------------------------------------------------------------------------------------
downloads_RDD1=downloads_RDD.where(downloads_RDD.r_os!="NA")
downloads_RDD1.createOrReplaceTempView('packages1')
# removed NA's
query_results = spark.sql('SELECT case when r_os like "linux%" THEN "Linux" ELSE case when r_os \
                          like "darwin%" Then "Darwin" Else "Mingw" END END as os, \
                          count(r_os) AS count FROM packages1 group by os order by count(*) desc')
insert_into_cassandra(query_results,"question2")

In [296]:
#---------------------------------    Question# 3   ---------------------------------------------------------
#     Top 10 (distinct) largest sized packages
#------------------------------------------------------------------------------------------------------------
query_results = spark.sql('SELECT package as package,max(CAST(size AS int)) as size FROM packages \
                          group by package order by size desc limit 10')
insert_into_cassandra(query_results,"question3")

In [297]:
#---------------------------------    Question# 4   ---------------------------------------------------------
#     What were the top 10 least popular (distinct) packages?
#------------------------------------------------------------------------------------------------------------
query_results = spark.sql('SELECT package as package,count(*) AS count FROM packages group by package\
                          order by count(*) limit 10')
insert_into_cassandra(query_results,"question4")

In [298]:
#---------------------------------    Question# 5   ---------------------------------------------------------
#     At what specific hour there are most of the download hits?
#------------------------------------------------------------------------------------------------------------
query_results = spark.sql('SELECT hour(time) as hour,count(*) AS count FROM packages group by\
                          hour(time) order by count(*) desc limit 1')
insert_into_cassandra(query_results,"question5")

In [299]:
#---------------------------------    Question# 6   ---------------------------------------------------------
#     What are the 5 most popular packages in UK? (Correction -> US)
#------------------------------------------------------------------------------------------------------------
query_results = spark.sql('SELECT package as package, count(*) AS count FROM packages where country="US" \
                          group by package order by count(*) desc limit 5')
insert_into_cassandra(query_results,"question6")

In [303]:
#---------------------------------    Question# 7   ---------------------------------------------------------
#     Show all packages downloaded by the machine with highest number of downloads?
#------------------------------------------------------------------------------------------------------------
query_results = spark.sql('SELECT ip_id, count(*) AS Count FROM packages group by ip_id order by count(*) desc')
highest_ip_id = query_results.take(1)[0][0]
query_results1 = spark.sql('SELECT package as package,count(1) as count from packages where\
                           ip_id="'+str(highest_ip_id)+'" group by package order by count(1) desc')
insert_into_cassandra(query_results1,"question7")

In [304]:
#---------------------------------    Question# 8   ---------------------------------------------------------
#     Show top three OSs that are most popular among the R programmers?
#------------------------------------------------------------------------------------------------------------
# checking from removed NA's data
query_results = spark.sql('SELECT case when r_os like "linux%" THEN "Linux" ELSE case when r_os \
                          like "darwin%" Then "Darwin" Else "Mingw" END END as os, COUNT(*) as count \
                          FROM packages1 GROUP BY OS ORDER BY COUNT(*) DESC limit 3')
insert_into_cassandra(query_results,"question8")

In [305]:
#---------------------------------    Question# 9   ---------------------------------------------------------
#     . How many R users still use 32 bit machines?
#------------------------------------------------------------------------------------------------------------
query_results = spark.sql('SELECT COUNT(r_arch) as users_32_bit FROM packages where r_arch="i386"')
insert_into_cassandra(query_results,"question9")

In [306]:
#---------------------------------    Question# 10   --------------------------------------------------------
#     Show total number of downloads by each country, use ascending order?
#------------------------------------------------------------------------------------------------------------
query_results = spark.sql('SELECT country as country,count(*) AS download_count FROM packages \
                          group by country order by count(*)')
insert_into_cassandra(query_results,"question10")